In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

import seaborn as sns
sns.set_style("whitegrid")

import random

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
# Load the data

lpmc = pd.read_csv('../../data/LPMC/trips.csv')

In [5]:
regions = lpmc.hh_borough.unique()

for r in tqdm(regions):

    dct = {'age': [], 'female': [], 'hh_borough': []}

    df_age = pd.read_csv('../../data/nomis_tmp/{}_age.csv'.format(r))
    df_age_sex = pd.read_csv('../../data/nomis_tmp/{}_age_sex.csv'.format(r))

    proba = df_age_sex['Males'] / df_age_sex['All']

    tmp_age = []
    tmp_sex = []

    for i in range(101):
        tmp_age.append(i*np.ones(df_age.iloc[i].values[0]))

        if i < 85:
            tmp_sex.append(1*np.ones(df_age_sex['Females'].iloc[i]))
            tmp_sex.append(0*np.ones(df_age_sex['Males'].iloc[i]))
        elif i == 85:

            females = df_age_sex['Females'].iloc[i]*[1]
            males = df_age_sex['Males'].iloc[i]*[0]
            tmp = np.concatenate((males, females))
            random.shuffle(tmp)
            tmp_sex.append(tmp)

    dct['age'] = np.concatenate(tmp_age).ravel().tolist()

    dct['female'] = np.concatenate(tmp_sex).ravel().astype(int).tolist()

    dct['hh_borough'] = len(dct['female'])*[r]

    df = pd.DataFrame(dct)

    df.to_csv('../../data/nomis/{}.csv'.format(r), index=False)

100%|██████████| 49/49 [00:30<00:00,  1.60it/s]


In [3]:
regions = {
    'Central London': ['City of London', 'Camden', 'Kensington and Chelsea', 'Islington', 'Westminster', 'Southwark', 'Lambeth'],
    'South London': ['Bromley', 'Croydon', 'Kingston upon Thames', 'Merton', 'Sutton', 'Wandsworth'],
    'East London': ['Barking and Dagenham', 'Bexley', 'Greenwich', 'Hackney', 'Havering', 'Lewisham', 'Newham', 'Tower Hamlets', 'Waltham Forest'],
    'North London': ['Barnet', 'Enfield', 'Haringey'],
    'West London': ['Brent', 'Ealing', 'Hammersmith & Fulham', 'Harrow', 'Hillingdon', 'Hounslow', 'Redbridge', 'Richmond upon Thames']
}

In [4]:
for r in regions:
    dfs = []
    for s in regions[r]:
        df = pd.read_csv('../../data/nomis/{}.csv'.format(s))
        df = df[df['age'] >=5]
        dfs.append(df)

    df = pd.concat(dfs)
    df['hh_region'] = r
    df.drop(columns=['hh_borough'], inplace=True)

    print('{} has {} rows'.format(r, len(df)))

    df.to_csv('../../data/nomis/{}.csv'.format(r), index=False)

Central London has 1314313 rows
South London has 1421383 rows
East London has 2077591 rows
North London has 854914 rows
West London has 1914245 rows
